# Shoe Classification Using CNNs

AIM: The aim of the project is to biuld a model using Convolutional Neural Networks, which correctly classifies images of mens shoes according to their type. The model will be constructed using Keras with TensorFlow as the backend.

In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Convolution2D, Flatten, MaxPooling2D, Dropout
import os
import numpy as np
from scipy import misc
from sklearn.model_selection import train_test_split
from keras.models import model_from_json
from keras.utils import np_utils
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sqlalchemy import create_engine
import pandas as pd
from PIL import Image

Using TensorFlow backend.


#### Model Architecture

In [138]:
engine=create_engine('postgresql://localhost:5432/capstone_project')
Data=pd.read_sql_table('mens_shoes_cleaned',con=engine)

In [141]:
## Define the shoe 'type' key words in order
Shoe_Types=[u'boots',u'loafers',u'derby',u'sneakers',u'brogues',u'ankle',u'oxfords',u'chelsea',
            u'monk',u'trainers',u'hightops',u'slippers',u'driving',u'gommino',u'drivers',u'chukka',
            u'sandals',u'slip',u'moccasin',u'lace']


## Creating a function that searches through the shoe type list, and if the type is found returns the type.
def shoe_type1(Inp_Str,Shoe_Types):
    for x in Shoe_Types:
        if x in Inp_Str:
            return x
## Creating a function that searches through the shoe type list, and if the type is found returns the type IF 
## the type has not already been assigned
def shoe_type2(Inp_Str,Type1,Shoe_Types):
    for x in Shoe_Types:
        if x!=Type1:
            if x in Inp_Str:
                return x

In [142]:
## Applying the functions above to the data set
Data['type_1']=Data.description.map(lambda x:shoe_type1(x,Shoe_Types))
Data['type_2']=Data.apply(lambda x:shoe_type2(x['description'],x['type_1'],Shoe_Types),axis=1)
## One shoe type I missed that needs to be aggregated is drivers and driving this is done below
Data.type_1=Data.type_1.map(lambda x: 'drivers' if x=='driving' else x)
Data.type_2=Data.type_2.map(lambda x: 'drivers' if x=='driving' else x)
Data.head()

,id,brand,retailer,description,image_url,image_url_alt,price,date,type_1,type_2
0,728,alexander mcqueen,Matches_Fashion,raised sole hightops leather trainers,http://assetsprx.matchesfashion.com/img/produc...,http://assetsprx.matchesfashion.com/img/produc...,395.0,2016-11-22,trainers,hightops
1,729,armando cabral,Matches_Fashion,lace up leather derby shoes,http://assetsprx.matchesfashion.com/img/produc...,http://assetsprx.matchesfashion.com/img/produc...,405.0,2016-11-22,derby,lace
2,730,bottega veneta,Matches_Fashion,intrecciato leather loafers,http://assetsprx.matchesfashion.com/img/produc...,http://assetsprx.matchesfashion.com/img/produc...,380.0,2016-11-22,loafers,None
3,731,armando cabral,Matches_Fashion,lizard effect leather ankle boots,http://assetsprx.matchesfashion.com/img/produc...,http://assetsprx.matchesfashion.com/img/produc...,530.0,2016-11-22,boots,ankle
4,732,burberry,Matches_Fashion,owen tread sole canvas boots,http://assetsprx.matchesfashion.com/img/produc...,http://assetsprx.matchesfashion.com/img/produc...,350.0,2016-11-22,boots,None


In [143]:
Data=Data[['id','type_1','type_2']]

In [145]:
Rename={'ankle':'boots',
        'chelsea':'boots',
        'gommino':'loafers',
        'moccasin':'loafers',
        'sandals':'loafers',
        'hightops':'sneakers',
        'slippers':'loafers',
        'monk':'formal',
        'drivers':'loafers',
        'oxfords':'formal',
        'brogues':'formal',
        'trainers':'sneakers',
        'derby':'formal',
        'loafers':'loafers',
        'boots':'boots',
        'sneakers':'sneakers'}

In [146]:
Data=Data[~Data.type_1.isin(['chukka','slip','lace'])]

In [148]:
# Replacing shoe type with dictionary
Data['Model_1']=Data.type_1.replace(Rename)

In [150]:
# Model 1 target variables
Data.Model_1.value_counts()

sneakers    5692
boots       1555
formal      1354
loafers     1276
Name: Model_1, dtype: int64

In [151]:
Colours=pd.read_csv('Colour_Features.csv')

In [183]:
# Make dataframe of all available images
Output=pd.merge(left=Data,right=Colours,left_on='id',right_on='ID',how='inner')

In [184]:
# Remove rows where shoe type is null
Output=Output[~Output.Model_1.isnull()]

In [185]:
Output.to_csv("Shoe_IDs.csv")

In [186]:
# Image file location
Root_Location='/Users/annacrawford/Desktop/Images_CNN/'

In [187]:
# Create list of all file names
files=list(Output.id)
files=[''.join(['_',str(x),'_a.jpg']) for x in files]

In [188]:
# Retrieve image id from filename in the order to be loaded
Image_Order=[x[1:-6] for x in files]

In [358]:
# Check all ids have an assosiated file

[i for i,x in enumerate(Image_Order) if x==7192]

[]

In [189]:
# Version 1 creating arrays from images and appending to list
Image_Data=[]
for x in files:
    try:
        dloc=''.join([Root_Location,x])
        im = Image.open(dloc)
        ar = misc.fromimage(im)
        Image_Data.append(ar)
        
    except:
        print 'Error processing image :',x

In [190]:
# Image size

Image_Data[0].shape

(200, 200, 3)

In [191]:
# Number of images

len(Image_Data)

9243

In [192]:
# Check that all images are the correct shape

[i for i,x in enumerate(Image_Data) if x.shape != (200, 200, 3)]

[]

In [193]:
# Creating array from list

X=np.array(Image_Data)

In [194]:
X.shape

(9243, 200, 200, 3)

In [195]:
# Define shoe type list

y=Output.Model_1
y.shape

(9243,)

In [196]:
# Classification list

np.unique(y)

array(['boots', 'formal', 'loafers', 'sneakers'], dtype=object)

In [197]:
# Saving X, y file to be used on AWS

with file('X_Data.npy', 'w') as Xoutfile:
    np.save(Xoutfile,X)
with file('y_Data.npy', 'w') as youtfile:    
    np.save(youtfile,y)

## Sneakers Subset

In [285]:
# Define Sneaker set

Sneakers=Output[Output.Model_1=='sneakers']

In [286]:
# Change sneaker type to sneaker of hightops, depending on description order

def sneakers_sub(type_1,type_2):
    if type_1=='hightops' or type_2=='hightops':
        return 'hightops'
    else:
        return 'sneakers'

In [287]:
Sneakers['Model_2']=Sneakers.apply(lambda x:sneakers_sub(x['type_1'],x['type_2']),axis=1)

/Users/annacrawford/anaconda/envs/Zaincorp/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [288]:
# Create file list from image ID

sneaker_files=list(Sneakers.id)
sneaker_files=[''.join(['_',str(x),'_a.jpg']) for x in sneaker_files]
sneaker_Image_Order=[x[1:-6] for x in sneaker_files]

In [289]:
# Open each image file appending array to list

sneaker_Image_Data=[]
for x in sneaker_files:
    try:
        dloc=''.join([Root_Location,x])
        im = Image.open(dloc)
        ar = misc.fromimage(im)
        sneaker_Image_Data.append(ar)
    except:
        print 'Error processing image :',x

In [290]:
# Check arrays are of the correct size
[i for i,x in enumerate(sneaker_Image_Data) if x.shape != (200, 200, 3)]

[]

In [291]:
# Number of shoes
y_sneaker=Sneakers.Model_2
y_sneaker.shape

(5361,)

In [292]:
# Classification list
np.unique(y_sneaker)

array(['hightops', 'sneakers'], dtype=object)

In [293]:
# Create array from list
X_sneaker=np.array(sneaker_Image_Data)

In [317]:
X_sneaker.shape

(5361, 200, 200, 3)

In [294]:
# Save X, y to file for use in AWS
with file('Sneaker_X_Data.npy', 'w') as Xoutfile:
    np.save(Xoutfile,X_sneaker)
with file('Sneaker_y_Data.npy', 'w') as youtfile:    
    np.save(youtfile,y_sneaker)

## Formal Shoes

In [295]:
# Define Formal shoe set
Formal=Output[Output.Model_1=='formal']

In [296]:
Formal['Model_2']=Formal.type_1

/Users/annacrawford/anaconda/envs/Zaincorp/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [297]:
# Create file names from image ID

formal_files=list(Formal.id)
formal_files=[''.join(['_',str(x),'_a.jpg']) for x in formal_files]
formal_Image_Order=[x[1:-6] for x in formal_files]

In [298]:
# Open each file, appending array to list
formal_Image_Data=[]
for x in formal_files:
    try:
        dloc=''.join([Root_Location,x])
        im = Image.open(dloc)
        ar = misc.fromimage(im)
        formal_Image_Data.append(ar)
    except:
        print 'Error processing image :',x

In [299]:
# Check each image has the correct shape

[i for i,x in enumerate(formal_Image_Data) if x.shape != (200, 200, 3)]

[]

In [300]:
# Number of images

y_formal=Formal.Model_2
y_formal.shape

(1249,)

In [301]:
# Classification list

np.unique(y_formal)

array([u'brogues', u'derby', u'monk', u'oxfords'], dtype=object)

In [302]:
X_formal=np.array(formal_Image_Data)

In [303]:
X_formal.shape

(1249, 200, 200, 3)

In [304]:
# Write to file for use in AWS

with file('Formal_X_Data.npy', 'w') as Xoutfile:
    np.save(Xoutfile,X_formal)
with file('Formal_y_Data.npy', 'w') as youtfile:    
    np.save(youtfile,y_formal)

## Boots

In [305]:
# Define Boots set

Boots=Output[Output.Model_1=='boots']

In [306]:
Boots.type_2.value_counts()

ankle       292
chelsea     247
lace        131
chukka       80
brogues      48
monk          9
derby         5
hightops      5
slip          4
gommino       2
sneakers      2
oxfords       1
Name: type_2, dtype: int64

In [307]:
# Define boots subcategory

Null_List=['oxfords','sneakers','gommino','slip','hightops','derby','monk','lace']
def boots_sub(type_1,type_2):
    if type_1!='boots':
        return type_1
    else:
        if type_2 in Null_List or type_2 ==None:
            return 'boots'
        else:
            return type_2

In [308]:
# apply subcategory function to dataset

Boots['Model_2']=Boots.apply(lambda x:boots_sub(x['type_1'],x['type_2']),axis=1)

/Users/annacrawford/anaconda/envs/Zaincorp/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [309]:
Boots.Model_2.value_counts()

boots      784
ankle      293
chelsea    249
chukka      80
brogues     48
Name: Model_2, dtype: int64

In [310]:
# Create file names from image ID

boots_files=list(Boots.id)
boots_files=[''.join(['_',str(x),'_a.jpg']) for x in boots_files]
boots_Image_Order=[x[1:-6] for x in boots_files]

In [311]:
# Open each file, appending its matrix to list

boots_Image_Data=[]
for x in boots_files:
    try:
        dloc=''.join([Root_Location,x])
        im = Image.open(dloc)
        ar = misc.fromimage(im)
        boots_Image_Data.append(ar)
    except:
        print 'Error processing image :',x

In [312]:
# Check to see if image has the correct shape
[i for i,x in enumerate(boots_Image_Data) if x.shape != (200, 200, 3)]

[]

In [313]:
# Number of boots

y_boots=Boots.Model_2
y_boots.shape

(1454,)

In [314]:
# Classification list
np.unique(y_boots)

array([u'ankle', 'boots', u'brogues', u'chelsea', u'chukka'], dtype=object)

In [315]:
X_boots=np.array(boots_Image_Data)

In [316]:
# Save to file for use in AWS

with file('Boots_X_Data.npy', 'w') as Xoutfile:
    np.save(Xoutfile,X_boots)
with file('Boots_y_Data.npy', 'w') as youtfile:    
    np.save(youtfile,y_boots)

## Loafers/Slippers

In [318]:
# Define loafer set

Loafers=Output[Output.Model_1=='loafers']

In [329]:
Loafers.type_2.value_counts()

slip        185
gommino      77
drivers      18
moccasin     10
lace          8
monk          4
brogues       4
sneakers      3
slippers      1
Name: type_2, dtype: int64

In [337]:
# Define loafer subcategory

Loaf_List=['drivers','slippers']

def loafers_sub(type_1,type_2):
    if type_1=='moccasin' or type_1=='gommino' :
        return 'drivers'
    
    if type_1!='loafers':
        return type_1
    else:
        if type_2=='moccasin'or type_1=='gommino':
            return 'drivers'
    
        if type_2 in Loaf_List:
            return type_2
        else:
            return 'loafers'

In [338]:
# Apply function to dataset

Loafers['Model_2']=Loafers.apply(lambda x:loafers_sub(x['type_1'],x['type_2']),axis=1)

/Users/annacrawford/anaconda/envs/Zaincorp/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [339]:
Loafers.Model_2.value_counts()

loafers     653
drivers     264
slippers    172
sandals      90
Name: Model_2, dtype: int64

In [341]:
# Create file names from image ID

loafers_files=list(Loafers.id)
loafers_files=[''.join(['_',str(x),'_a.jpg']) for x in loafers_files]
loafers_Image_Order=[x[1:-6] for x in loafers_files]

In [348]:
# open image appending array to list

loafers_Image_Data=[]
for x in loafers_files:
    try:
        dloc=''.join([Root_Location,x])
        im = Image.open(dloc)
        ar = misc.fromimage(im)
        loafers_Image_Data.append(ar)
    except:
        print 'Error processing image :',x

In [349]:
#  Check size of images is correct

[i for i,x in enumerate(loafers_Image_Data) if x.shape != (200, 200, 3)]

[]

In [350]:
# Number of shoes

y_loafers=Loafers.Model_2
y_loafers.shape

(1179,)

In [351]:
# Classification list

np.unique(y_loafers)

array(['drivers', 'loafers', u'sandals', u'slippers'], dtype=object)

In [352]:
X_loafers=np.array(loafers_Image_Data)

In [353]:
X_loafers.shape

(1179, 200, 200, 3)

In [354]:
# Save to file for use in AWS

with file('Loafers_X_Data.npy', 'w') as Xoutfile:
    np.save(Xoutfile,X_loafers)
with file('Loafers_y_Data.npy', 'w') as youtfile:    
    np.save(youtfile,y_loafers)

### The code below was copied into a .py script and run on AWS.

In [215]:
# Convert arrays to float

X=X.astype("float32")

In [237]:
# Transform target variable to categorical

Enc=LabelEncoder()
y= Enc.fit_transform(y)

In [238]:
# Define class list

CLass_list = Enc.classes_

array(['boots', 'chelsea', 'formal', 'hightops', 'loafers', 'monk',
       'sandals', 'sneakers'], dtype=object)

In [203]:
# save class list to file

with file('Class_list.npy', 'w') as Classfile:
    np.save(Classfile,Class_List)

In [239]:
# Train test split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)

In [198]:
model = Sequential() # Define sequential model

# The first convolutional layer takes the input of the colour image, which is 200 x 200 in size, and applies 
# 32 different filters (each 2 X 2) in order to detect features in the shoes. The filters used are chosen, like
# the weights in the network, by backpropagation. Each filter is convolved over the input matrix to produce
# another image by means of matrix multiplication. An activation function is then applied element 
# wise to the resulting image. The Relu activation function is defined as the max(0, x), in essence eliminating
# negative values.

model.add(Convolution2D(nb_filter = 32, nb_col = 3, nb_row = 3, input_shape = (200, 200, 3), activation = 'relu' ))

# MaxPooling is then applied to the convoluted image. The purpose of MaxPooling is to down sample the image, thus
# reducing the dimentionality of the feature space and allowing for assumptions to be made about features contained 
# in the sub-regions binned. A 2 x 2 matrix convolves over the image matrix and takes the maximum value contained
# within that region. The advantaged of MaxPooling is that it prevents the network from over-fitting and also
# reduces the computational time in training.

model.add(MaxPooling2D(pool_size=(2, 2)))

# Dropout randomly selects 25% of the input units, setting them to zero. This also prevents over-fitting.
model.add(Dropout(0.25))

model.add(Convolution2D(nb_filter = 32, nb_col = 3, nb_row = 3, activation = 'relu' ))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Flatten, flattens the matrix into an array for input into the network.
model.add(Flatten())

# The initial weights of the nodes in the network are randomly assigned from the Uniform distribution, which are then
# updated in the backpropagation process.
model.add(Dense(128, init = 'uniform', activation = 'relu'))
model.add(Dense(128, init = 'uniform', activation = 'relu'))
model.add(Dropout(0.25))

# Like in most networks, the final layer contains the activation function softmax. The softmax function squashes a 
# K-dimentional vector z of arbitrary real values, to a K-dimentional vector of real values between 0 and 1 that 
# sum to 1. The output of the softmax function will be the probability distribution over the different classes.
model.add(Dense(8, init = 'uniform', activation = 'softmax')) # 11 classes

# To evaluate the weights in the networks we must specify the loss function, and to optimise the weights we must
# specify the optimiser. Because this is a classification problem, the loss function used will be binary_crossentropy.
# The optimiser adam was chosen for no other reason than efficientsy. The evlauation metric used to judge
# the networks performance will be accuracy

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

# Fit model
model.fit(X_train, keras.utils.np_utils.to_categorical(y_train), nb_epoch = 20, batch_size = 32, verbose = 0)

Evaluate model
score = model.evaluate(X_test, keras.utils.np_utils.to_categorical(y_test))
print score

In [167]:
# Save model along with model weights

model_json=model.to_json()
with open("model.json",'w') as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")

In [168]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk
